# OCT + iPhone + Samsung con validación cruzada

In [3]:
import os
import cv2
import seaborn as sn 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report  
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import VGG19
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

**PREPARACIÓN DE LOS DATOS**

In [4]:
color = 'rgb' #'grayscale' o 'rgb'

escala = 150

In [5]:
def cargar_datos(path1, path2, escala = escala, color = color):
    
    EMD = os.listdir(path1)
    NO_EMD = os.listdir(path2)
    
    data = []
    labels = []

    for i in EMD:   
        image=tf.keras.preprocessing.image.load_img(path1+'/'+i, color_mode= color, 
        target_size= (escala, escala))
        image=np.array(image)
        data.append(image)
        labels.append(1)
    for i in NO_EMD:   
        image=tf.keras.preprocessing.image.load_img(path2+'/'+i, color_mode= color, 
        target_size= (escala, escala))
        image=np.array(image)
        data.append(image)
        labels.append(0)
        
    data = np.array(data)
    labels = np.array(labels)
    
    return data, labels

In [6]:
dataset_iphone = cargar_datos('Datos preprocesados INP EMD/iPhone/EMD', 'Datos preprocesados INP EMD/iPhone/NO EMD')

In [7]:
dataset_samsung = cargar_datos('Datos preprocesados INP EMD/Samsung/EMD', 'Datos preprocesados INP EMD/Samsung/NO EMD')

In [8]:
dataset = cargar_datos('Datos preprocesados INP EMD/OCT/EMD', 'Datos preprocesados INP EMD/OCT/NO EMD')

In [9]:
X = np.concatenate((dataset[0],dataset_samsung[0]))

X = np.concatenate((X,dataset_iphone[0]))

y = np.concatenate((dataset[1],dataset_samsung[1]))

y = np.concatenate((y,dataset_iphone[1]))

**DEFINICIÓN DEL MODELO TRANSFER LEARNING CON FIT() / SCORE() / PREDICT()**

In [10]:
class KerasModel(BaseEstimator, ClassifierMixin):
    
    dataset_iphone = cargar_datos('Datos preprocesados INP EMD/iPhone/EMD', 'Datos preprocesados INP EMD/iPhone/NO EMD')
    dataset_samsung = cargar_datos('Datos preprocesados INP EMD/Samsung/EMD', 'Datos preprocesados INP EMD/Samsung/NO EMD')
    dataset = cargar_datos('Datos preprocesados INP EMD/OCT/EMD', 'Datos preprocesados INP EMD/OCT/NO EMD')
    
    X = np.concatenate((dataset[0],dataset_samsung[0]))

    X = np.concatenate((X,dataset_iphone[0]))

    y = np.concatenate((dataset[1],dataset_samsung[1]))

    y = np.concatenate((y,dataset_iphone[1]))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
    
    def fit(self, X_train, y_train):
        
        train_labels_categorical = to_categorical(y_train, num_classes=2)
        
        X_train = preprocess_input(X_train) 
        y_train = preprocess_input(y_train)
        
        ## Loading VGG16 model
        base_model = VGG19(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
        base_model.trainable = False ## Not trainable weights
        
        flatten_layer = layers.Flatten()
        dense_layer_1 = layers.Dense(50, activation='relu')
        dense_layer_2 = layers.Dense(20, activation='relu')
        prediction_layer = layers.Dense(2, activation='softmax')

        model = models.Sequential([
            base_model,
            flatten_layer,
            dense_layer_1,
            dense_layer_2,
            prediction_layer
        ])
        
        classes = np.unique(y_train)
        class_weights = class_weight.compute_class_weight('balanced', classes=classes, y=y_train)
        dic_class_weights = {0:class_weights[0], 1:class_weights[1]}
        
        model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )

        es = EarlyStopping(monitor='val_accuracy', mode='max', patience=20,  restore_best_weights=True)

        model.fit(x=X_train,y=train_labels_categorical, epochs=200, validation_split=0.2, batch_size=32, callbacks=[es], class_weight=dic_class_weights)

        self.model = model
        
        return self
    
    def score(self, X_test, y_test):
        
        predictions = self.model.predict(X_test)
        y_pred = []
        
        for e in predictions:
            if e[0]>e[1]:
                y_pred.append(0)
            else:
                y_pred.append(1)
                
        return accuracy_score(y_test, y_pred)
    
    def predict(self, X_test):
                
        return self.model.predict(X_test)

**VALIDACIÓN CRUZADA**

Evaluar una puntuación mediante validación cruzada.

Devuelve una matriz de puntuaciones del estimador para cada ejecución de la validación cruzada (en este caso hacemos 5).

In [11]:
def val_score(X, y):
    
    clf = KerasModel()
    scores = cross_val_score(clf, X, y)
    
    print("Con cross_val_score() hemos obtenido ")
    print(scores)
    
    print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [29]:
def val_predict(X, y):
    
    clf = KerasModel()
    y_pred = cross_val_predict(clf, X, y)
    
    y_pred_bin = []

    for e in y_pred:
        if e[0]>e[1]:
            y_pred_bin.append(0)
        else:
            y_pred_bin.append(1)
    
    roc_score = roc_auc_score(y, y_pred_bin, multi_class='ovr')
    print("El valor AUC ROC para esta predicción es ")
    print(roc_score)
    
    f1 = f1_score(y, y_pred_bin, average='weighted')
    print("El F1 score para esta predicción es ")
    print(f1)
    
    matriz = confusion_matrix(y, y_pred_bin)
    print("La matriz de confución de esta predicción es ")
    print(matriz)

## VGG19

In [12]:
val_score(X, y)

Epoch 1/200
7/7 [==============================] - 10s 1s/step - loss: 179.8325 - accuracy: 0.5744 - val_loss: 0.0133 - val_accuracy: 1.0000
Epoch 2/200
7/7 [==============================] - 9s 1s/step - loss: 88.5414 - accuracy: 0.6564 - val_loss: 0.8537 - val_accuracy: 0.6939
Epoch 3/200
7/7 [==============================] - 11s 2s/step - loss: 52.9605 - accuracy: 0.7795 - val_loss: 6.1062 - val_accuracy: 0.0204
Epoch 4/200
7/7 [==============================] - 11s 2s/step - loss: 49.4852 - accuracy: 0.7487 - val_loss: 0.0328 - val_accuracy: 0.9796
Epoch 5/200
7/7 [==============================] - 13s 2s/step - loss: 56.2904 - accuracy: 0.7282 - val_loss: 3.0346 - val_accuracy: 0.4082
Epoch 6/200
7/7 [==============================] - 12s 2s/step - loss: 15.5431 - accuracy: 0.9128 - val_loss: 1.5218 - val_accuracy: 0.6531
Epoch 7/200
7/7 [==============================] - 11s 2s/step - loss: 18.6595 - accuracy: 0.8410 - val_loss: 1.7756 - val_accuracy: 0.5714
Epoch 8/200
7/7 [===

Epoch 16/200
7/7 [==============================] - 11s 2s/step - loss: 1.1100 - accuracy: 1.0000 - val_loss: 1.2106 - val_accuracy: 0.6122
Epoch 17/200
7/7 [==============================] - 11s 2s/step - loss: 0.9068 - accuracy: 1.0000 - val_loss: 1.2870 - val_accuracy: 0.6122
Epoch 18/200
7/7 [==============================] - 11s 2s/step - loss: 0.7834 - accuracy: 1.0000 - val_loss: 1.2031 - val_accuracy: 0.6531
Epoch 19/200
7/7 [==============================] - 11s 2s/step - loss: 0.7181 - accuracy: 1.0000 - val_loss: 1.3269 - val_accuracy: 0.6122
Epoch 20/200
7/7 [==============================] - 11s 2s/step - loss: 0.6396 - accuracy: 1.0000 - val_loss: 1.3503 - val_accuracy: 0.6122
Epoch 21/200
7/7 [==============================] - 11s 2s/step - loss: 0.5715 - accuracy: 1.0000 - val_loss: 1.2992 - val_accuracy: 0.6531
Epoch 22/200
7/7 [==============================] - 11s 2s/step - loss: 0.5445 - accuracy: 1.0000 - val_loss: 1.2986 - val_accuracy: 0.6122
Epoch 23/200
7/7 [==

2/2 [==============================] - 3s 1s/step
Con cross_val_score() hemos obtenido 
[0.73770492 0.73770492 0.70491803 0.59016393 0.73770492]
0.70 accuracy with a standard deviation of 0.06


In [30]:
val_predict(X, y)

Epoch 1/200
7/7 [==============================] - 11s 1s/step - loss: 230.8083 - accuracy: 0.4769 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 2/200
7/7 [==============================] - 9s 1s/step - loss: 74.1827 - accuracy: 0.7077 - val_loss: 0.4145 - val_accuracy: 0.8776
Epoch 3/200
7/7 [==============================] - 11s 2s/step - loss: 33.3845 - accuracy: 0.8051 - val_loss: 2.1068 - val_accuracy: 0.4082
Epoch 4/200
7/7 [==============================] - 11s 2s/step - loss: 21.3176 - accuracy: 0.8359 - val_loss: 1.2490 - val_accuracy: 0.5714
Epoch 5/200
7/7 [==============================] - 11s 2s/step - loss: 10.1933 - accuracy: 0.9231 - val_loss: 2.2493 - val_accuracy: 0.4082
Epoch 6/200
7/7 [==============================] - 11s 2s/step - loss: 12.1895 - accuracy: 0.8718 - val_loss: 0.0803 - val_accuracy: 0.9796
Epoch 7/200
7/7 [==============================] - 11s 2s/step - loss: 8.7545 - accuracy: 0.9538 - val_loss: 1.3683 - val_accuracy: 0.5714
Epoch 8/200
7/7 [====

Epoch 16/200
7/7 [==============================] - 11s 2s/step - loss: 0.4817 - accuracy: 0.9949 - val_loss: 3.1248 - val_accuracy: 0.5102
Epoch 17/200
7/7 [==============================] - 11s 2s/step - loss: 0.1252 - accuracy: 1.0000 - val_loss: 2.6492 - val_accuracy: 0.5510
Epoch 18/200
7/7 [==============================] - 11s 2s/step - loss: 0.0846 - accuracy: 1.0000 - val_loss: 2.5903 - val_accuracy: 0.5510
Epoch 19/200
7/7 [==============================] - 11s 2s/step - loss: 0.0752 - accuracy: 1.0000 - val_loss: 2.7376 - val_accuracy: 0.5510
Epoch 20/200
7/7 [==============================] - 11s 2s/step - loss: 0.0516 - accuracy: 1.0000 - val_loss: 2.8935 - val_accuracy: 0.5306
Epoch 21/200
2/2 [==============================] - 3s 1s/step
Epoch 1/200
7/7 [==============================] - 12s 2s/step - loss: 152.2585 - accuracy: 0.5231 - val_loss: 0.6236 - val_accuracy: 0.9184
Epoch 2/200
7/7 [==============================] - 11s 2s/step - loss: 87.8115 - accuracy: 0.682